In [1]:
import sys,  time
import ipywidgets as widget
from IPython.display import display
import numpy as np
import cv2
from PIL import Image as PIL_Image
from io import BytesIO
cv2.startWindowThread()
def img_to_png(ima, cvt=None):
    if cvt:
        ima = cv2.cvtColor(ima, cvt)
    im = PIL_Image.fromarray(ima)
    bio = BytesIO()
    im.save(bio, format='png')
    return bio.getvalue()

In [3]:
import dlib
sd = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
fr = cv2.face.createEigenFaceRecognizer()
fd = dlib.get_frontal_face_detector()
r = fd(img)[0]

In [5]:
cap = cv2.VideoCapture(0)
cap.set(3,1920)
cap.set(4,1080)
while True:
    ret, img0 = cap.read()
    img = cv2.resize(img0, (640,360))
    H, W = img.shape[:2]    
    keycode = cv2.waitKey(1) & 0xff
    if  keycode == ord('q'):
        break
    #imgray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #ret,thresh = cv2.threshold(imgray,127,255,0)
    #im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    #cv2.drawContours(img, contours, -1, (0,255,0), 1)
    
    #img = edges = cv2.Canny(img,100,200)
    #img[:,:,0] += edges*128
    """ 
    mask = np.zeros(img.shape[:2],np.uint8)
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
    rect = (50,50,450,290)
    cv2.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    img = img*mask2[:,:,np.newaxis]
    """
    rl = fd(img)
    for r in rl:
        shape = sd(img, r)
        poly = np.int32([(x.x, x.y) for x in shape.parts()])*2
        img0 = cv2.polylines(img0, np.int32([poly[:17]]) , 0, (0,255,255))
        img0 = cv2.polylines(img0, np.int32([poly[17:22]]) , 0, (255,0,255))
        img0 = cv2.polylines(img0, np.int32([poly[22:27]]) , 0, (255,0,255))
        img0 = cv2.polylines(img0, np.int32([poly[27:31]]) , 0, (0,255,0))
        img0 = cv2.polylines(img0, np.int32([poly[30:36]]) , 1, (0,255,0))
        img0 = cv2.polylines(img0, np.int32([poly[36:42]]) , 1, (255,255,255))
        img0 = cv2.polylines(img0, np.int32([poly[42:48]]) , 1, (255,255,255))
        img0 = cv2.polylines(img0, np.int32([poly[48:60]]) , 1, (255,255,0))
        img0 = cv2.polylines(img0, np.int32([poly[60:68]]) , 1, (255,255,0))
        #img = cv2.rectangle(img, (r.left(),r.top()), (r.right(),r.bottom()),  (255,255,0))
    
    img0 = cv2.flip(img0, 1)
    cv2.imshow('frame', img0)

    #time.sleep(0.3)
cv2.destroyWindow('frame')
cap.release()

In [155]:
img0.shape

(720, 1280, 3)

In [7]:
def seg1():   
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
    # sure background area
    sure_bg = cv2.dilate(opening,kernel,iterations=3)
    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
    ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)
    # Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)
    # Add one to all labels so that sure background is not 0, but 1
    markers = markers+1
    # Now, mark the region of unknown with zero
    markers[unknown==255] = 0
    markers = cv2.watershed(img,markers)
    img[markers == -1] = [255,0,0]

In [65]:
while True:
    cv2.imshow('im2', img2)
    keycode = cv2.waitKey(1) & 0xff
    if  keycode == ord('q'):
        break

In [85]:
shape.part(1)

point(278, 157)

[(278, 132),
 (278, 157),
 (278, 181),
 (281, 205),
 (290, 226),
 (304, 244),
 (323, 258),
 (345, 267),
 (368, 270),
 (390, 267),
 (406, 257),
 (420, 243),
 (431, 226),
 (438, 208),
 (443, 187),
 (445, 168),
 (447, 150),
 (309, 108),
 (321, 95),
 (338, 90),
 (355, 91),
 (371, 98),
 (392, 101),
 (407, 99),
 (422, 101),
 (434, 109),
 (440, 122),
 (381, 119),
 (381, 131),
 (382, 143),
 (383, 156),
 (360, 174),
 (370, 176),
 (379, 179),
 (387, 178),
 (394, 176),
 (326, 124),
 (335, 121),
 (345, 121),
 (353, 125),
 (345, 126),
 (335, 126),
 (398, 130),
 (408, 128),
 (417, 130),
 (424, 136),
 (416, 135),
 (407, 133),
 (342, 210),
 (357, 202),
 (369, 197),
 (377, 200),
 (384, 199),
 (393, 204),
 (400, 214),
 (391, 220),
 (382, 223),
 (373, 223),
 (366, 222),
 (354, 219),
 (347, 210),
 (369, 206),
 (376, 207),
 (384, 207),
 (395, 213),
 (383, 212),
 (375, 213),
 (367, 212)]